In [ ]:
#Time Intensity measurments
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import seaborn as sns
import openpyxl
import xlsxwriter
import random
from numpy import mean
import statistics as s


<u>The data from EyeQ are structured column-wise
<ul>
    <li> Product I Session I Replica I Sequence I Q1_Melting I Q2_1s I Q2_2s I ... I Q2_180s
        <li> Melting of data based on Judge (Panelist), Product and Session = df_melt
           
</ul>

In [ ]:

root = r'C:\Users\burka\OneDrive\Desktop\global TI_.xlsx'
df = pd.read_excel(root)

##Do you want to delete unncesseray Columns such as Replica, Sequence or Comments##
columns = list(df.columns.values) 
df = df.drop(columns = ["Replica",'Sequence',str(columns[-1])]) 
columns = list(df.columns.values) 

##COLUMN NAMES CHANGE ACCORDING TO TIME SEQUENCES##
#time_interval = int(input("Was ist das Zeitintervall zwischen den Punkten?"))
time_interval = 1 ##Frequency of data Recording
Pre_Int = False #If only certain frequency ranges
Anz_Pre_Int = 3  #Number of pre-defined intervals
Zeit_zw_Pre_Int = 10 #Time b/t pre-defined intervals
non_int_col = 4 # number of columns before starting the time - when df Replica and Sequence is dropped
decay_alt = True 


###APPROACH##
#1 - RENAME ALL COLUMNS  
#A.df - drop unnecessary columns B. df_melt for sns 

def list2(nomencl,targetarray):
    for i in nomencl:
        i = []
        targetarray.append(i)

        
#Start column that needs to be changed is 4 - #RENAME ALL COLUMNS         
column_new = []
arr = df.columns.values

if Pre_Int == True:
    for i in range(Anz_Pre_Int):
        k = (i+1)*Zeit_zw_Pre_Int #Interval Vordefiniert - Laufvariable
        column_new.append(str(k))
    for j in range(len(arr[non_int_col:])-Anz_Pre_Int):
        k = (Anz_Pre_Int)*Zeit_zw_Pre_Int + (j+1) * time_interval
        column_new.append(str(k))
else:
    for j in range(len(arr[non_int_col:])):
        k = (j+1) * time_interval
        column_new.append(str(k))
    
        
arr = df.columns.values
arr[non_int_col-1] = 'Stop Time'
arr[non_int_col:] = column_new
df3 = df.drop(columns = ['Stop Time']) ##needed for visualisation of Raw Data


#RAW DATA Visualization#
df_melt = pd.melt(df3, id_vars=["Judge","Product","Session"]) #Herunterbrechen nach Variablen Judge,Product,Session

<u>Raw Data Visualization using sns.catplot and save


In [ ]:
g = sns.catplot(data=df_melt, # from your Dataframe
                   col="Product", # Make a subplot in columns for each variable in "animal"
                   col_wrap=2, # Maximum number of columns per row 
                   x="variable", # on x-axis make category on the variable "variable" (created by the melt operation)
                   y="value", # The corresponding y values
                   hue="Judge", # color according to the column gender
                   kind="point", # the kind of plot, the closest to what you want is a stripplot, 
                   legend_out=True, # let the legend inside the first subplot.
                   )

g.set_xlabels("Time")
g.set_ylabels("Intensity")
g.set_titles(template="{col_name}") # otherwise it's "animal = dog", now it's just "dog"
sns.despine(trim=True) # trim the axis.

g.savefig('RawData.png')

# EXPORT OF PLOT
export = False

if export == True:
    #root = input("Please copy pace the path input, where you want to store your data")
    new_name = input('Please specify the name of your export data (dont forget .jpg):')
    new_path = "%s" % ((root))  # Output
    g.savefig('%s%s' %((new_path, new_name)))

<u>Raw Data Visualization using sns.catplot and save
<ul>
    <li> create specific time-marks in new df

In [ ]:
sf = df
                        
tstart = []  
Istart = []
tmax = []
Imax = []
tasc = []
Iasc = []
tmelt = []
tend = []
Iend = []


k = [[] for l in range(sf.shape[0])]

for j in range(sf.shape[0]): #sf.shape[0] = Anz Reihen
    #a-f sind nur Laufvariablen
    transit = pd.to_numeric(sf.iloc[j,non_int_col:])
    #MAX VALUE 
    a = sf.iloc[j,non_int_col:].max() #4 b/c four non int columns before
    Imax.append(a)
    b = transit.idxmax()
    tmax.append(b)

    #START
    c = [n for n, i in enumerate(transit) if i>0][0]
    tstart.append(sf.columns[c+non_int_col])
    Istart.append(transit[c])

    # DESCENDING LAST MAX VALUE OR FIRST TIME SINGLE MAX VALUE IS LEFT
    if pd.Series(transit).value_counts().get(max(transit)) > 1:
        d = [n for n, i in enumerate(transit) if i == max(transit)][-1] 
    else:
        if decay_alt == False:
            d = [n for n, i in enumerate(transit) if i == max(transit)][0]+1  ###THE +1 means appending 1
        else:
            d = [n for n, i in enumerate(transit) if i == max(transit)][0]
    
    tasc.append(sf.columns[d+non_int_col])
    Iasc.append(transit[d])

    #T-END --> if ZERO start conditioning
    cond = True in (ele == 0 for ele in transit)
    for i in range(sf.shape[1]-4):
        if transit[i] == 0:
            if i > int(tmax[j]):
                k[j].append(i)
    
    if cond == True:
        if  [n for n, i in enumerate(transit) if i == 0][0] != 0:
            s = [n for n, i in enumerate(transit) if i == 0][0]
            tend.append(sf.columns[s+non_int_col])
        elif pd.Series(transit).value_counts().get(0) > 1:
            s = [n for n, i in enumerate(transit) if i == 0][1]
            tend.append(sf.columns[s+non_int_col])
        else: ##NUR ALS NOTLOESUNG DA NIEMALS BEI 0 ---> kann später gelöscht werden
            tend.append(160)
    else: 
        tend.append(160)
    
    Iend = [0 for k in range(sf.shape[0])] #bis hier Notlösung   

    #T-SCHMELZ ZEITPUNKT - ALWAYS CONTROL 
    tmelt.append(sf.iloc[j,3])


tend = []
for i in range(len(k)):
    tend.append(float(k[i][0]))


tstart = [float(x) for x in tstart]
tmax = [float(x) for x in tmax]
tasc = [float(x) for x in tasc]
tmelt = [float(x) for x in tmelt]
#tend = [float(x) for x in tend]

sf_final = df.drop(df.iloc[:, non_int_col-1:], axis = 1)

##COLUMN ERWEITERUNG##
sf_final['tstart'] = tstart
sf_final['Istart'] = Istart
sf_final['tmax'] = tmax
sf_final['Imax'] = Imax
sf_final['tdesc'] = tasc
sf_final['Idesc'] = Iasc
sf_final['tmelt'] = tmelt
sf_final['tend'] = tend
sf_final['Iend'] = Iend 

liste = ['tstart_g','Istart_g','tmax_g','Imax_g','tdesc_g','Idesc_g','tmelt_g','tend_g','Iend_g']

for n in range(len(liste)): #3 b/c of Judge,Product,Session
    pend = [[] for x in range(len(sf['Product'].unique()))]
    for i in range(sf.shape[0]):
        for j in range(len(sf['Product'].unique())):
            if sf.iloc[i,1] == sf['Product'].unique()[j]:
                pend[j].append(sf_final.iloc[i,non_int_col-1+n])
    sf_final[liste[n]] = pd.NaT
    
    for i in range(sf.shape[0]):
        for j in range(len(sf['Product'].unique())):
             if sf.iloc[i,1] == sf['Product'].unique()[j]:
                    sf_final.iloc[i,-1] = mean(pend[j])
            

<u>Normalization in Intensity (y-Axis) of Data 


In [ ]:
#3 - Intensity Normed based on Imax,i and Imax,g for each product - in_df (and _melt)

#in_df = intensity normed data frame 
in_df = {'Judge':[str(sf['Judge'].tolist()[i]) for i in range(sf.shape[0])],
         'Product':[str(sf['Product'].tolist()[i]) for i in range(sf.shape[0])]}
in_df = pd.DataFrame(in_df)

columns = list(df.columns.values) #COLUMN NAMES FOR NEW DF 'First Values = 4'

for j in range(len(columns)-non_int_col): #There the 15 and 6 are fixed from sf_final and independent on df above
    in_df[columns[non_int_col+j]] = pd.NaT
    for i in range(sf.shape[0]):
        in_df.iloc[i,2+j] = float((sf.iloc[i,non_int_col+j] * sf_final.iloc[i,15]) / sf_final.iloc[i,6])

       
        
#RAW DATA Visualization#
in_df_melt = pd.melt(in_df, id_vars=["Judge","Product"]) #Herunterbrechen nach Variablen Judge,Product,Session

<u>Normalization in Time-Axis (x-Axis) of Data 


In [ ]:
4 - Normalize in_df also in t-direction to have product given same tstart,max,dec and end

## rows = number of rows in df * number of time iterations 
last = ['Stop Time']
num_time_it = list(df.columns.values).index(last[0]) ##equivalent to 4 in beginning  -1 

## For the new df -> first 1Judge then through all time iterations, then next.....
df_adapted_melt = {'Judge':[str(df['Judge'].tolist()[i]) for i in range(df.shape[0])
                            for j in range(df.shape[1]-num_time_it-1)],
                   'Product':[str(df['Product'].tolist()[i]) 
                              for i in range(df.shape[0]) for j in range(df.shape[1]-num_time_it-1)],
                  'Session':[str(df['Session'].tolist()[i]) 
                              for i in range(df.shape[0]) for j in range(df.shape[1]-num_time_it-1)]}
df_adapted_melt = pd.DataFrame(df_adapted_melt)


## NOW NORMALIZE TIME 

## TWO WAYS OF ITERATING THROUGH DATA EITHER VIA COMPARING if Judge,Prod und Session = gleich, aber das benötigt 
# drei Laufvariablen, deshalb aufgrund der Strukturierung von oben wird Jede Kombi Judge,Prod und Session
# chronologisch durchgegangen nachdem alle zeiten abgegangen sind

tstarti = list(sf_final.columns.values).index('tstart')
tmaxi = list(sf_final.columns.values).index('tmax')
tdeci = list(sf_final.columns.values).index('tdesc') 
tendi = list(sf_final.columns.values).index('tend') 
tstartg = list(sf_final.columns.values).index('tstart_g')
tmaxg = list(sf_final.columns.values).index('tmax_g')
tdecg = list(sf_final.columns.values).index('tdesc_g')
tendg = list(sf_final.columns.values).index('tend_g')

time_list = list(df.columns.values)[num_time_it+1:]
transit_t = []

### TIME NORMALIZATION
for j in range(sf.shape[0]):
    for i in range(len(list(df.columns.values)[num_time_it+1:])):
        if float(time_list[i]) <= sf_final.iloc[j,tmaxi]:  ### NORMALIZE ALL -MAX VALUE
            if float(time_list[i]) < sf_final.iloc[j,tstarti]: ##Falls bspw 10 = time_list[i] < time_start 
                t = 0
                transit_t.append(t)
            elif sf_final.iloc[j,tmaxi] == sf_final.iloc[j,tstarti]:
                t = sf_final.iloc[j,tmaxg]
                transit_t.append(t)
            else:
                t = ((sf_final.iloc[j,tmaxg] - sf_final.iloc[j,tstartg]) / 
                     (sf_final.iloc[j,tmaxi] - sf_final.iloc[j,tstarti])) * (float(time_list[i]) - sf_final.iloc[j,tstarti]) + sf_final.iloc[j,tstartg]
                transit_t.append(round(t,2))
        elif sf_final.iloc[j,tmaxi] < int(time_list[i]) and int(time_list[i]) < sf_final.iloc[j,tdeci]: ### NORMALIZE ALL VALUES BT MAX AND DEC
            t = ((sf_final.iloc[j,tdecg] - sf_final.iloc[j,tmaxg]) / 
                 (sf_final.iloc[j,tdeci] - sf_final.iloc[j,tmaxi])) * (float(time_list[i]) - sf_final.iloc[j,tmaxi]) + sf_final.iloc[j,tmaxg]
            transit_t.append(round(t,2))
        elif sf_final.iloc[j,tdeci] <= float(time_list[i]): ## NORMALIZE ALL OTHERS 
            t = ((sf_final.iloc[j,tendg] - sf_final.iloc[j,tdecg]) / 
                 (sf_final.iloc[j,tendi] - sf_final.iloc[j,tdeci])) * (float(time_list[i]) - sf_final.iloc[j,tdeci]) + sf_final.iloc[j,tdecg]
            transit_t.append(round(t,2))


df_adapted_melt['Time Normalized'] = transit_t
df_adapted_melt['Int Normalized'] = pd.NaT
transit_I = []

for k in range(df.shape[0]): ##ADD INTENSITIES TO TIME
    for l in range(len(list(df.columns.values)[num_time_it+1:])):
        df_adapted_melt.iloc[l+k*len(list(df.columns.values)[num_time_it+1:]),4] = in_df.iloc[k,2 + l]  #2 because Judge/Product needs to be neglected



In [ ]:
#5 - Create new x-axis dataframe on the basis of molten df 
#and corresponding intensities based on linear interpolation b/t nearest neighbours

products = sorted(sf_final['Product'].unique())

def globaltime(n):
    itera =  ['tstart_g','tmax_g','tdesc_g','tend_g']
    arr = [[] for k in range(len(products))]
    for j in range(sf.shape[0]):
        for i in range(len(products)):
            if sf_final.iloc[j,1] == str(products[i]):
                arr[i].append(sf_final[itera[n]][j])            
    for i  in range(len(products)):
        del arr[i][1:]
    trans = []
    for i in range(len(products)):
        trans.append(float(arr[i][0]))
    return trans

start_mean = globaltime(0)
max_mean = globaltime(1)
dec_mean = globaltime(2)
end_mean = globaltime(3)


## NEW TIME RANGES DEFINE ##
# r = max_m[i] - start_m[i], r2 = desc_m[i] - max_m[i] # nur 1L pro Produkt
nu_int = 20


def interval(anz_p): #with k index = number of product *0 = P01...
    arr = [[] for m in range(anz_p)]
    for m in range(anz_p):
        for i in range(nu_int):
            interval = max_mean[m] - start_mean[m]
            add = round(interval * i/nu_int + start_mean[m],1) ##max_i - min_i * i & number of int + min_i
            arr[m].append(add)
        for i in range(nu_int):
            interval = dec_mean[m] - max_mean[m]
            add = round(interval * i/nu_int + max_mean[m],1)
            arr[m].append(add)
        for i in range(nu_int):
            interval = end_mean[m] - dec_mean[m]
            add = round(interval * i/nu_int + dec_mean[m],1)
            arr[m].append(add) 
    
    return arr


def interval2(anz_p):
    arr = [[] for m in range(anz_p)]
    for m in range(anz_p):
        for i in range(3*nu_int):
            interval = end_mean[m] - start_mean[m]
            add = round(interval * i/(3*nu_int) +start_mean[m],1)
            arr[m].append(add)
    return arr



## There new list is possibly defined
if decay_alt == False:
    t_int_neu = interval(len(products))
else: 
    t_int_neu = interval2(len(products))

    


In [ ]:
##create corresponding list w/ talt, tneu siehe oben, Ialt und dann basierend darauf neue listen
t_alt = df_adapted_melt['Time Normalized']
t_alt = [round(float(t_alt[i]),1) for i in range(len(t_alt))] #numpy float to float 
I_alt = df_adapted_melt['Int Normalized']
I_alt = [round(float(I_alt[i]),1) for i in range(len(I_alt))] #numpy float to float and round


##PRINCIPLE OF INTERPOLATION BASED ON LINEAR INTERPOLATION B/T NEAREST NEIGHBOURS - 
#hence based on product (from df_adapted_melt) new lists with high low is created and always only with 
#the loop of  (len(time interval old))

##FOR THE EQUATION to find new Int to tneu_int(i) 
## I'' = tneu_int(t) - thigh / (thigh-tlow) * (Ihigh- Ilow) + Ilow
## if thigh = tlow -> I'' ? = Ilow 
## if thigh not existent --> what then ?

def time_hoch(numberel):
    t_alt = df_adapted_melt['Time Normalized']

    time_high = [[] for m in range(sf.shape[0])]
    for r in range(sf.shape[0]):
        for i in range(len(products)):
            t_neu = t_int_neu[i]
            for k in range(len(t_neu)):
                t_alt = [round(float(t_alt[j]),1) for j in range(len(t_alt))] #numpy float to float 
                t_alt_ = t_alt[0+numberel*r:numberel+numberel*r]
                for m,l in enumerate(t_alt_):
                    if df_adapted_melt.iloc[m+numberel*r,1] == str(products[i]):
                        if l >= t_neu[k]:
                            time_high[r].append(l)
                            del time_high[r][1+k:] 
    return time_high


time_list = list(df.columns.values)[num_time_it+1:]
time_high = time_hoch(len(time_list))

time_low = [[] for m in range(sf.shape[0])]
trans = [[] for f in range(len(t_int_neu[0]))]
for r in range(sf.shape[0]):
    for i in range(len(products)):
        t_neu = t_int_neu[i]
        for k in range(len(t_neu)):
            t_alt = [round(float(t_alt[i]),2) for i in range(len(t_alt))]
            t_alt_ = t_alt[0+len(time_list)*r:len(time_list)+len(time_list)*r]
            for m,l in enumerate(t_alt_):
                if df_adapted_melt.iloc[m+len(time_list)*r,1] == str(products[i]):
                    if l <= t_neu[k]:
                        trans[k].append(l)
                if len(trans[k]) > 1:
                    del trans[k][:-1]
    time_low[r] = [float(trans[i][0]) for i in range(len(t_int_neu[0]))]

    
int_high = [[] for k in range(sf.shape[0])]
for k in range(sf.shape[0]):
    time_high_ = time_high[k]
    for j in range(len(time_high_)):
        for i in range(len(time_list)):
            if round(df_adapted_melt.iloc[i+len(time_list)*k][3],1) == time_high_[j]:
                int_high[k].append(df_adapted_melt.iloc[i+len(time_list)*k][4])

int_low = [[] for k in range(sf.shape[0])]
for k in range(sf.shape[0]):
    time_low_ = time_low[k]
    for j in range(len(time_low_)):
        for i in range(len(time_list)):
            if round(df_adapted_melt.iloc[i+len(time_list)*k][3],1) == time_low_[j]:
                int_low[k].append(df_adapted_melt.iloc[i+len(time_list)*k][4])
   
    
### INTENSITY NORMALIZED
## A. I'' = tneu_int(t) - thigh / (thigh-tlow) * (Ihigh- Ilow) + Ilow
## B. if thigh = tlow -> I'' ? = Ilow 
## C. if thigh not existent --> then append values = last and then tlow = thigh -> then 


In [ ]:
intensity_final = []##FINAL INTENSITITES
for k in range(sf.shape[0]):
    for i in range(len(t_int_neu[0])):
        if len(time_high[k]) < len(t_int_neu[0]):
            while len(time_high[k]) < len(t_int_neu[0]): ##CASE C. always check to bring t_high to same level evy with the while
                time_high[k].insert(-1,time_high[k][-1])
        elif len(int_high[k]) < len(t_int_neu[0]):   
            while len(int_high[k]) < len(t_int_neu[0]):
                int_high[k].insert(-1,int_high[k][-1])
        elif len(int_low[k]) < len(t_int_neu[0]):
            while len(int_low[k]) < len(t_int_neu[0]):
                int_low[k].insert(-1,int_low[k][-1])
        for j in range(len(products)):
            if df_adapted_melt.iloc[len(time_list)*k,1] == str(products[j]):
                if time_high[k][i] == time_high[k][i]:
                    intensity_final.append(int_low[k][i])
                else:
                    intensity_final.append(((t_int_neu[j][i]-time_low[k][i]) / (time_high[k][i] - time_low[k][i])) * 
                                           (int_high[k][i] - int_low[k][i])
                              + int_low[k][i])


In [ ]:
import statistics as s
#6- create final molten df with normalized and time-scaled avg data 
#7 - take means / stdv 


## For the new df -> first 1Judge then through all time iterations, then next.....
df_final_norm = {'Judge':[str(df['Judge'].tolist()[i]) for i in range(df.shape[0])
                            for j in range(len(t_int_neu[0]))],
                   'Product':[str(df['Product'].tolist()[i]) 
                              for i in range(df.shape[0]) for j in range(len(t_int_neu[0]))],
                  'Session':[str(df['Session'].tolist()[i]) 
                              for i in range(df.shape[0]) for j in range(len(t_int_neu[0]))]}
df_final_norm = pd.DataFrame(df_final_norm)
df_final_norm['Time Normalized'] = pd.NaT
                    
liste = []

for k in range(sf.shape[0]):
    for i in range(len(t_int_neu[0])):
        for j in range(len(products)):
             if df_final_norm.iloc[i+len(t_int_neu[0])*k,1] == str(products[j]):
                    liste.append(t_int_neu[j][i])

df_final_norm['Time Normalized'] = liste
df_final_norm['Intensity Normalized'] = intensity_final

df_final_norm


#create trans lists to average all data (incl stdv)
trans = [[[] for j in range(len(products))] for m in range(len(t_int_neu[0]))]
#len(t_int_neu[0])
for k in range(sf.shape[0]):
    for i in range(len(t_int_neu[0])):
        for j in range(len(products)):
            if df_final_norm.iloc[i+len(t_int_neu[0])*k,1] == str(products[j]):
                if t_int_neu[j][i] == df_final_norm.iloc[i+len(t_int_neu[0])*k,3]:
                    trans[i][j].append(df_final_norm.iloc[i+len(t_int_neu[0])*k,4])


final_means = [[[] for j in range(len(products))] for m in range(len(t_int_neu[0]))]
final_stdv = [[[] for j in range(len(products))] for m in range(len(t_int_neu[0]))]
for i in range(len(t_int_neu[0])):
    for j in range(len(products)):
        final_means[i][j] = mean(trans[i][j])
        final_stdv[i][j] = s.pstdev(trans[i][j])

df_final_norm



In [ ]:
##Addition of 0 value at 0 in final_means[0][1] and [0][0]

#import numpy as np
#only if zero value integrated
#final_means = np.insert(final_means, 0, 0, 0)
#final_stdv = np.insert(final_stdv, 0, 0, 0)
#t_int_neu[0].insert(0, 0)
#t_int_neu[1].insert(0,0)
lines_melt = [[] for k in range(len(sf_final['Product'].unique()))]
for j in range(sf_final.shape[0]):
    for i in range(len(sf_final['Product'].unique())):
        if sorted(sf_final['Product'].unique())[i] == sf_final['Product'][j]:
            lines_melt[i].append(sf_final['tmelt_g'][j])
for i in range(len(sf_final['Product'].unique())):
    lines_melt[i] = lines_melt[i][-1]
    
fig, ax = plt.subplots(figsize=(15, 15))
fig.suptitle(' Normierte Kurven ', fontsize=30) 
products = sorted(sf_final['Product'].unique())


for i in range(len(products)):
    trans_means = [[] for range in range(len(products))]
    trans_stdv = [[] for range in range(len(products))]
    for j in range(len(t_int_neu[0])):
        trans_means[i].append(final_means[j][i])
        trans_stdv[i].append(final_stdv[j][i])
        
    plt.subplot(4,2,i+1)
    
    plt.plot(t_int_neu[i],trans_means[i], '--bo', label ='line with marker')

    #plt.scatter(t_int_neu[i], trans_means[i], color='b')
    plt.axvline(x=float(lines_melt[i]))
    plt.xlabel('t_norm')
    plt.ylabel('I_norm')
    plt.title('%s' %products[i], fontsize = 10) 
    #plt.errorbar(t_int_neu[i], a[i], yerr=b[i], fmt="o", color="r")  
    fig.tight_layout()
plt.show()
fig.savefig('Normierte Kurven.jpg')

In [ ]:
df_final_mean = {'Product': [products[i]  for j in range(len(t_int_neu[0])) for i in range(len(products)) ],
                'Time':[t_int_neu[j][i] for i in range(len(t_int_neu[0])) for j in range(len(products))],
                 'Int Means':[final_means[i][j] for i in range(len(t_int_neu[0])) for j in range(len(products))],
                 'Int Stdv':[final_stdv[i][j] for i in range(len(t_int_neu[0])) for j in range(len(products))]      
                }

df_final_mean = pd.DataFrame(df_final_mean)
df_final_mean

In [ ]:

### EXPORT OF DATA IN ONE EXCEL SHEET ###


name = r'globalTI2_real_.xlsx'

#df = RAW DATA
#sf_final = AVERAGED NUMBER
#in_df = NORMALIZED INTENSITY
#df_adpated_melt = Normalized T - I molten and not time interval adjusted
#df_final_normed 
#df_final_means

# g.savefig('dummy.png')

with pd.ExcelWriter('%s%s' %(root[:-10],name)) as writer:

    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df.to_excel(writer, sheet_name="Raw Data", index=False)
    #dummy.to_excel(writer,sheet_name='Img Raw Data',index=False)
    #worksheet = writer.sheets['Raw Data']
    #worksheet.insert_image('X2','RawData.png')
    sf_final.to_excel(writer, sheet_name="Averaged Numbers", index=False)
    #in_df.to_excel(writer, sheet_name="Normalized Intensity", index=False)
    #worksheet = writer.sheets['Normalized Intensity']
    #worksheet.insert_image('X2','NormalizedInt.png')
    #df_adapted_melt.to_excel(writer, sheet_name ='Molten Norm T-I', index = False)
    #worksheet = writer.sheets['Molten Norm T-I']
    #worksheet.insert_image('G2','Molten_TimeInt.png')
    #df_final_norm.to_excel(writer, sheet_name = 'Molten RAW data Normed',index = False)
    df_final_mean.to_excel(writer,sheet_name = 'AVERAGED FINAL DATA',index =False)
    worksheet = writer.sheets['AVERAGED FINAL DATA']
    worksheet.insert_image('G2','Normierte Kurven.jpg')